<a href="https://colab.research.google.com/github/aashikrasool/Coefficient-Based-Data-Generator/blob/main/correlation_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ***Data preprocess***

In [ ]:
#this process is for mount data from colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
crop_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Crop_recommendation.csv")

In [ ]:
crop_data.head()

In [ ]:
crop_data['label']

In [ ]:
# Initialize an empty dictionary to store data for each crop type
crop_dict = {}

In [ ]:


import pandas as pd


# Create global variables for each crop type
for label in crop_data['label'].unique():
    globals()[f"{label}_data"] = crop_data[crop_data['label'] == label]

# Print the names of these variables
print("Created variable names:")
for label in crop_data['label'].unique():
    crop_var_name = f"{label}_data"
    if crop_var_name in globals():
        print(crop_var_name)


In [ ]:
#producing class wise
rice_correlation = rice.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have numeric columns in the 'rice' DataFrame
rice_correlation =jute_data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Create a heatmap using seaborn
sns.heatmap(rice_correlation, annot=True, cmap='coolwarm', fmt=".2f")

# Display the plot
plt.title('Correlation Matrix - Rice Data')
plt.show()

In [ ]:



import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

real_data = pomegranate_data
origianl_df=pomegranate_data
# Extract correlation coefficients from real data
correlation_matrix = real_data.corr()
correlation_coefficient = correlation_matrix.iloc[0, 1]

In [ ]:
real_data.head()

In [ ]:
correlation_matrix

In [ ]:
# Instantiate the LabelEncoder
label_encoder = LabelEncoder()
real_data['label'] = label_encoder.fit_transform(real_data['label'])

In [ ]:
real_data['label']

Build GAN


In [ ]:


# Generate synthetic data with correlation
def generate_data(num_samples, correlation_coefficient):
    np.random.seed(42)
    data = np.random.rand(num_samples, 8)
    correlated_feature = correlation_coefficient * data[:, 0] + 0.2 * np.random.rand(num_samples)
    data[:, 1] = correlated_feature
    return data

# Define a custom self-attention layer
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def build(self, input_shape):
        self.Wq = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.Wk = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.Wv = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)

    def call(self, x):
        Q = tf.matmul(x, self.Wq)
        K = tf.matmul(x, self.Wk)
        V = tf.matmul(x, self.Wv)

        attention_weights = tf.nn.softmax(tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(K)[-1], tf.float32)))
        output = tf.matmul(attention_weights, V)

        return output

# Define the generator model
def build_generator(latent_dim, output_dim):
    model = models.Sequential()

    # Input layer
    model.add(layers.Dense(32, input_dim=latent_dim, activation='relu'))

    # Add self-attention layer
    model.add(SelfAttention())

    # Additional hidden layers
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))

    # Output layer
    model.add(layers.Dense(output_dim, activation='linear'))  # Change output_dim to 7

    return model
# Define the discriminator model
def build_discriminator(input_dim):
    model = models.Sequential()
    model.add(layers.Dense(32, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Build and compile the GAN model with correlation loss
def build_gan(generator, discriminator, correlation_coefficient):
    discriminator.trainable = False
    gan_model = models.Sequential()
    gan_model.add(generator)
    gan_model.add(discriminator)

    # Add correlation loss to the generator's loss function
    gan_model.compile(loss=['binary_crossentropy', 'mean_squared_error'], optimizer='adam')
    return gan_model

def train_gan(generator, discriminator, gan_model, data, epochs, batch_size, latent_dim, correlation_coefficient):
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_data = data[idx]
        fake_data = generator.predict(np.random.randn(batch_size, latent_dim))
        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))
        d_loss_real = discriminator.train_on_batch(real_data, labels_real)
        d_loss_fake = discriminator.train_on_batch(fake_data, labels_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator with correlation loss
        noise = np.random.randn(batch_size, latent_dim)
        labels_gan = np.ones((batch_size, 1))
        correlation_target = correlation_coefficient * noise[:, 0]  # Target correlation for the generated data
        g_loss = gan_model.train_on_batch(noise, [labels_gan, correlation_target])

        # Print progress
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")

# Set parameters
num_samples = len(real_data)
latent_dim = 10
data_shape = 8  # Change this based on the number of features in your data
epochs = 1000 # Increase the number of epochs for better training
batch_size = 64

# Split data into train and test sets
train_data, test_data = train_test_split(real_data, test_size=0.2, random_state=42)

# Extract correlation coefficients from real training data
train_correlation_matrix = train_data.corr()
train_correlation_coefficient = train_correlation_matrix.iloc[0, 1]
# Print the data type of train_data.values
print(train_data.values.dtype)
# Generate correlated data using only the training data
num_samples_train = len(train_data)
synthetic_data_train = generate_data(num_samples_train, train_correlation_coefficient)

# Denormalize synthetic data if necessary

# Display synthetic data for training
synthetic_df_train = pd.DataFrame(synthetic_data_train, columns=[f'Feature_{i+1}' for i in range(data_shape)])
print("Synthetic Data (Training):")
print(synthetic_df_train.head())

# Build and compile models
generator = build_generator(latent_dim, data_shape)
discriminator = build_discriminator(data_shape)
gan_model = build_gan(generator, discriminator, train_correlation_coefficient)

# Train the GAN on the training data
train_gan(generator, discriminator, gan_model, train_data.values, epochs, batch_size, latent_dim, train_correlation_coefficient)

# Evaluate correlation between generated and real data after training
num_samples_test = len(test_data)
synthetic_data_test = generator.predict(np.random.randn(num_samples_test, latent_dim))
synthetic_df_test = pd.DataFrame(synthetic_data_test, columns=[f'Feature_{i+1}' for i in range(data_shape)])

correlation_matrix_synthetic = synthetic_df_test.corr()
correlation_coefficient_synthetic = correlation_matrix_synthetic.iloc[0, 1]

print(f"Correlation Coefficient between Real and Synthetic Data: {correlation_coefficient_synthetic}")
print(type(train_data.values))
# # Display synthetic data for training
# synthetic_df_train = pd.DataFrame(synthetic_data_train, columns=[f'Feature_{i+1}' for i in range(data_shape)])
# print("Synthetic Data (Training):")
# print(synthetic_df_train.head())

# Extract column names from the original data
original_column_names = real_data.columns

# Display synthetic data for training
synthetic_df_train = pd.DataFrame(synthetic_data_train, columns=original_column_names)
# synthetic_df_train = synthetic_df_train *100
print("Synthetic Data (Training):")
print(synthetic_df_train.head())

# # Scale synthetic data to match the value range of the original data
# for column in original_column_names:
#     min_val = real_data[column].min()
#     max_val = real_data[column].max()
#     synthetic_df_train[column] = synthetic_df_train[column] * (max_val - min_val) + min_val

# Save synthetic data to CSV with the same column names and value range as the original data
synthetic_df_train.to_csv('/content/drive/MyDrive/Colab Notebooks/sdata_train_pomegranate.csv', index=False)
print("Synthetic Data (Training) saved to synthetic_data_train1.csv")
# Saving the model in the TensorFlow SavedModel format
# generator.save('/content/drive/MyDrive/Colab Notebooks/sdata_generator_model.h5')

In [ ]:
# Calculate the min and max values for each feature in the original data
original_mins = real_data.min()
original_maxs = real_data.max()

# Scale the synthetic data to the range observed in the original data
for column in synthetic_df_train.columns:
    if column != 'label':  # Skip the 'label' column
        # Scale the column to the min and max values of the original data
        synthetic_df_train[column] = synthetic_df_train[column] * (original_maxs[column] - original_mins[column]) + original_mins[column]

# Set the 'label' column to 'rice'
synthetic_df_train['label'] = 'pomegranate'

# Save the scaled synthetic data to a CSV file
synthetic_df_train.to_csv('/content/drive/MyDrive/Colab Notebooks/s_data_train_pomegranate.csv', index=False)
print("Synthetic Data (Training) saved to synthetic_data_train.csv")

In [ ]:
s=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/synthetic_data_train_apple.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for column in real_data.columns:
    if column != 'label':  # Assuming 'label' is a categorical column
        plt.figure(figsize=(10, 4))
        sns.histplot(real_data[column], color='blue', kde=True, label='Original')
        sns.histplot(synthetic_df_train[column], color='orange', kde=True, label='Synthetic', alpha=0.7)
        plt.title(f'Histogram for {column}')
        plt.legend()
        plt.show()

In [ ]:
for column in real_data.columns:
    if column != 'label':  # Assuming 'label' is a categorical column
        plt.figure(figsize=(10, 4))
        sns.histplot(real_data[column], color='green', kde=True, label='Original')
        sns.histplot(synthetic_df_train[column], color='purple', kde=True, label='Synthetic', alpha=0.7)
        plt.title(f'Histogram for {column}')
        plt.legend()
        plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have numeric columns in the 'rice' DataFrame

s=s.corr()
# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Create a heatmap using seaborn
sns.heatmap(s, annot=True, cmap='coolwarm', fmt=".2f")

# Display the plot
plt.title('Correlation Matrix - Rice Data')
plt.show()

In [ ]:
# import pandas as pd
# tgan_data=pd.read_csv("/content/TABGAN.csv")

In [ ]:
# correlation_matrix =tgan_data.corr()
# correlation_coefficient = correlation_matrix.iloc[0, 1]